In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
import time
import json

# Ustawienie urządzenia: GPU, jeśli jest dostępne, w przeciwnym razie CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 1. Przygotowanie niestandardowego datasetu do wczytywania obrazów i cen
class CarImageDataset(Dataset):
    def __init__(self, json_file, image_dir, transform=None):
        self.data = pd.read_json(json_file, lines=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx]['img_local'])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        price = self.data.iloc[idx]['price']
        return {'image': image, 'price': torch.tensor(price, dtype=torch.float32)}

# 2. Definicja modelu CNN (ResNet18) do regresji
class CarPriceCNN(nn.Module):
    
    def __init__(self):
        super(CarPriceCNN, self).__init__()
        
        # Ekstraktor cech obrazu - ResNet (bez warstwy końcowej klasyfikacji)
        self.cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Usuwamy ostatnią warstwę FC
        self.cnn_out_features = 512 

        # Połączenie wyjść CNN i MLP
        self.fc = nn.Sequential(
            nn.Linear(self.cnn_out_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # Wyjście przewidujące cenę
        )

    def forward(self, image):
        # Przepuszczenie obrazu przez CNN
        cnn_features = self.cnn(image)
        cnn_features = cnn_features.view(cnn_features.size(0), -1)


        # Predykcja ceny
        output = self.fc(cnn_features)
        return output

# 3. Przygotowanie transformacji obrazu
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 4. Wczytanie danych i DataLoader
image_dir = 'data_img/data_img'
json_file = 'train_data_with_car_type.json'
dataset = CarImageDataset(json_file=json_file, image_dir=image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 5. Inicjalizacja modelu, optymalizatora i funkcji straty
model = CarPriceCNN().to(device)  # Przeniesienie modelu na GPU
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 6. Trenowanie modelu na GPU z pomiarem czasu
num_epochs = 30
for epoch in range(num_epochs):
    start_time = time.time()  # Start pomiaru czasu dla epoki

    model.train()
    running_loss = 0.0
    for batch in dataloader:
        images = batch['image'].to(device)    # Przeniesienie obrazów na GPU
        prices = batch['price'].to(device)    # Przeniesienie cen na GPU

        optimizer.zero_grad()

        # Przewidywanie i obliczanie straty
        outputs = model(images).squeeze()
        loss = criterion(outputs, prices)

        # Aktualizacja wag
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Obliczenie średniego błędu i czasu dla epoki
    avg_loss = running_loss / len(dataloader)
    epoch_duration = time.time() - start_time
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Duration: {epoch_duration:.2f} seconds")

# 7. Zapis wytrenowanego modelu (opcjonalnie)
torch.save(model.state_dict(), 'car_price_cnn.pth')


In [ ]:
def evaluate_model(model, dataloader):
    model.eval()  # Ustawienie modelu w tryb ewaluacji
    total_loss = 0.0
    criterion = nn.MSELoss()
    
    with torch.no_grad():  # Wyłączenie gradientów dla oszczędności pamięci
        for batch in dataloader:
            images = batch['image'].to(device)
            prices = batch['price'].to(device)
            
            outputs = model(images).squeeze()
            loss = criterion(outputs, prices)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Test Loss (MSE): {avg_loss:.4f}")
    return avg_loss

# 5. Ładowanie wytrenowanego modelu
model = CarPriceCNN().to(device)
model.load_state_dict(torch.load('car_price_cnn.pth'))

# 6. Przygotowanie danych testowych i DataLoadera
image_dir = 'data_img/data_img'
json_file = 'test_data_with_car_type.json'
test_dataset = CarImageDataset(json_file=json_file, image_dir=image_dir, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 7. Uruchomienie ewaluacji
evaluate_model(model, test_dataloader)